In [1]:
# Generate the video for the env with a trained model

In [2]:
import sys
sys.path.insert(0, "../")
sys.path.insert(0, "../torchdrivesim/")

In [3]:
import numpy as np
import gymnasium as gym
from stable_baselines3 import PPO, SAC, TD3
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, SubprocVecEnv

import torchdriveenv
from torchdriveenv.env_utils import load_waypoint_suite_data, load_rl_training_config, load_replay_data

/home/kezhang/work/torchdriveenv/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Commercial access denied and fallback to check for academic access.


In [4]:
# model = SAC.load("models/BaselineAlgorithm.sac_1712996522/model")
model = SAC.load("best_model")

In [5]:
data = load_waypoint_suite_data("data/validation_cases.yml")
# data = load_replay_data("offline_datasets/replay_data_20240416-1323")
env_config = load_rl_training_config("env_configs/rl_training.yml").env
env_config.render_mode = "video"
env_config.record_episode_data = True
env_config.record_replay_data = False
env_config.ego_only = True

In [6]:
def make_env():
    env = gym.make('torchdriveenv-v0', args={'cfg': env_config, 'data': data})
    env = Monitor(env)  
    return env

In [7]:
def main(cfg, model):
    env = DummyVecEnv([make_env])
    env = VecFrameStack(env, n_stack=3, channels_order="first")
    obs = np.expand_dims(env.reset()[0], 0)
    states = None
    episode_starts = [True]
    i = 0

    while True:
        actions, states = model.predict(
         obs,  
         state=states,
         episode_start=episode_starts,
         deterministic=True,
        )
        obs, reward, done, info = env.step(actions)      
        if done:
            break
#         print(reward)
        if i > 200:
            break
            
        print(i)
        print(reward)
        i += 1
        
    env.close()

In [8]:
env_config

EnvConfig(ego_only=True, max_environment_steps=200, use_background_traffic=True, terminated_at_infraction=True, seed=None, simulator=TorchDriveConfig(renderer=RendererConfig(backend='default', render_agent_direction=True, left_handed_coordinates=True, highlight_ego_vehicle=True), single_agent_rendering=False, collision_metric=<CollisionMetric.nograd: 'nograd'>, offroad_threshold=0.5, left_handed_coordinates=True), render_mode='video', video_filename='rendered_video.mp4', video_res=1024, video_fov=500, record_episode_data=True, record_replay_data=False, use_blame=False)

In [9]:
main(env_config, model)

INFO:torchdriveenv.gym_env:seed: 1978534166
INFO:torchdriveenv.gym_env:    def get_reward(self):
        x = self.simulator.get_state()[..., 0]
        y = self.simulator.get_state()[..., 1]
        psi = self.simulator.get_state()[..., 2]

        d = math.dist((x, y), (self.last_x, self.last_y)) if (self.last_x is not None) and (self.last_y is not None) else 0
        distance_reward = 1 if d > 0.5 else 0
        psi_reward = (1 - math.cos(psi - self.last_psi)) * (-20.0) if (self.last_psi is not None) else 0
        if self.check_reach_target():
            reach_target_reward = 10
            self.reached_waypoint_num += 1
        else:
            reach_target_reward = 0
        r = torch.zeros_like(x)
        r += reach_target_reward + distance_reward + psi_reward
        return r

/home/kezhang/work/torchdriveenv/.venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtyp

0
[-0.00730151]
1
[-0.0225876]
2
[-0.02143036]
3
[-0.00944757]
4
[-0.01997643]
5
[-0.0109514]
6
[-6.6875714e-08]
7
[-0.00855412]
8
[-0.07618333]
9
[-0.08970609]
10
[-0.11081897]
11
[-0.13941954]
12
[-0.14198174]
13
[-0.192349]
14
[-0.20777602]
15
[-0.28102365]
16
[-0.23289339]
17
[-0.2588211]
18
[-0.33384916]
19
[-0.42436516]
20
[-0.38032988]
21
[-0.48655272]
22
[-0.31578812]
23
[-0.24591722]
24
[-0.629881]
25
[-0.4759671]
26
[-0.2824856]
27
[-0.49961773]
28
[-0.5558471]
29
[-0.51446205]
30
[-0.5445033]
31
[-0.5906564]
32
[-0.32162565]
33
[-0.24144101]
34
[-0.6585021]
35
[0.34208632]
36
[-0.07751256]
37
[-0.41276678]
38
[9.398676]
39
[-0.00646672]
40
[0.26079482]
41
[0.23132022]
42
[0.14392282]
43
[0.07796362]
44
[-0.06210753]
45
[-0.07132635]
46
[-0.16060705]
47
[-0.05581806]
48
[-0.13411602]
49
[0.5733513]
50
[0.05166202]
51
[0.05520337]
52
[0.19247894]
53
[0.25627983]
54
[-0.6213564]
55
[-0.51778454]
56
[-0.5810021]
57
[-0.6055601]
58
[0.33951598]
59
[-0.19542494]
60
[0.23979697]
61